In [1]:
seconds_to_minutes = lambda x: x * 60
run_every_minutes = seconds_to_minutes(10)

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

MAIN_PATH = 'speed_test/'
TEMP_PATH = f'{MAIN_PATH}temp/'
WORKING_PATH= f'{MAIN_PATH}raw_data/'
CHECK_POINT = f'{MAIN_PATH}_checkpoint'

SCHEMA = 'speed_test'
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {SCHEMA} ")

BRONZE_TABLE = f'{SCHEMA}.speed_test_logs'
spark.sql(f"""
  CREATE EXTERNAL TABLE IF NOT EXISTS {BRONZE_TABLE} 
  (log_timestamp Timestamp
  ) USING delta
  LOCATION '{WORKING_PATH.replace('speed_test','speed_test.db')}'
  TBLPROPERTIES (delta.enableChangeDataFeed = true)
""")

24/04/13 20:04:55 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/13 20:04:55 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/04/13 20:04:56 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/04/13 20:04:56 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore hadoop@127.0.1.1


DataFrame[]

In [3]:
spark.sql(
    "set spark.databricks.delta.changeDataFeed.timestampOutOfRange.enabled = true;"
)
spark.sql("set SQLConf.ADAPTIVE_EXECUTION_ENABLED.key= true")

DataFrame[key: string, value: string]

In [4]:
from pyspark.sql.types import *
import pyspark.sql.functions as F

In [5]:
streaming_logs_schema = StructType([StructField("download",
                    StructType([StructField("bandwidth", LongType(), True),
                                StructField("bytes", LongType(), True),
                                StructField("elapse", LongType(), True),
                                StructField("latency", StringType(), True)])
                                      , True),
             StructField("interface", StringType(), True),
             StructField("isp",StructType(), True),
             StructField("ping", StructType(), True),
             StructField("result", StructType([StructField("id", StringType(), True),
                                StructField("url", StringType(), True),
                                StructField("persisted", BooleanType(), True),
                               ]), True),
             StructField("server", StringType(), True),
             StructField("timestamp", TimestampType(), True),
             StructField("type", StringType(), True),
             StructField("upload", 

                    StructType([StructField("bandwidth", LongType(), True),
                                StructField("bytes", LongType(), True),
                                StructField("elapse", LongType(), True),
                                StructField("latency", StringType(), True)]), True)
                                ])


streaming_logs_sdf = spark.readStream.schema(streaming_logs_schema)\
    .json(WORKING_PATH)\
            .select([F.col('result').getItem('id').alias('test_id'),
                F.col("timestamp").alias('log_timestamp'),
                F.col('download').getItem('bytes').alias('download_bytes'),
                F.col('upload').getItem('bytes').alias('upload_bytes')])\
    .withColumn('download_Mbytes',F.col('download_bytes')/1000000)\
        .withColumn('upload_Mbytes',F.col('upload_bytes')/1000000)\
        .withColumn('log_file',F.input_file_name())




In [6]:
spark.sql(f'SELECT * FROM {BRONZE_TABLE} LIMIT 3').toPandas()

24/04/13 20:04:57 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
24/04/13 20:04:59 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,log_timestamp,test_id,download_bytes,upload_bytes,download_Mbytes,upload_Mbytes,log_file
0,2024-04-13 19:30:49,1e5cfd5f-1171-4734-9502-4bbed0000931,481158816,18186880,481.158816,18.18688,file:///home/hadoop/deltalake/speed_test/raw_d...
1,2024-04-13 17:41:02,f1e2c384-026b-4f07-a240-bc9a6f0dff99,566126008,22625000,566.126008,22.62500,file:///home/hadoop/deltalake/speed_test/raw_d...
2,2024-04-13 19:08:48,34d24d69-24c2-427a-b185-f9546fa596cb,572540648,8101560,572.540648,8.10156,file:///home/hadoop/deltalake/speed_test/raw_d...


In [7]:
streaming_logs_sdf.writeStream.format('delta')\
.outputMode("append")\
.option("checkpointLocation", CHECK_POINT)\
.trigger(processingTime='0 seconds')\
.option("overwriteSchema", "true")\
.option("mergeSchema", "true")\
.toTable(BRONZE_TABLE)

24/04/13 20:05:01 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [8]:
import subprocess, sys
import threading
import time
from datetime import datetime as dt
import os
from glob import glob
os.makedirs(WORKING_PATH,exist_ok=True)


def speed_test():
    
    def move_temp_to_raw():
        files = glob(f'{TEMP_PATH}*.json')
        list(map(lambda x: os.rename(x, x.replace(TEMP_PATH,WORKING_PATH)),files ))
        
    while True:
        now = dt.now()
        file_name =  f"log_{now.strftime('%Y_%m_%d_%H_%M_%s')}.json"
        subprocess.run(f'touch {TEMP_PATH}{file_name} && speedtest --format=json >> {TEMP_PATH}{file_name} && mv  {TEMP_PATH}{file_name} {WORKING_PATH}{file_name}', shell = True, executable="/bin/bash")
        move_temp_to_raw()
        time.sleep(run_every_minutes)
        


thread = threading.Thread(target=speed_test)
thread.start()


In [9]:
spark.sql(f'SELECT * FROM {BRONZE_TABLE} LIMIT 3').toPandas()

[2024-04-13 20:05:02.150] [error] Limit reached:

Speedtest CLI. Too many requests received. To maintain a fair and stable environment, please review and adjust the frequency of your requests.


,log_timestamp,test_id,download_bytes,upload_bytes,download_Mbytes,upload_Mbytes,log_file
0,2024-04-13 19:30:49,1e5cfd5f-1171-4734-9502-4bbed0000931,481158816,18186880,481.158816,18.18688,file:///home/hadoop/deltalake/speed_test/raw_d...
1,2024-04-13 17:41:02,f1e2c384-026b-4f07-a240-bc9a6f0dff99,566126008,22625000,566.126008,22.62500,file:///home/hadoop/deltalake/speed_test/raw_d...
2,2024-04-13 19:08:48,34d24d69-24c2-427a-b185-f9546fa596cb,572540648,8101560,572.540648,8.10156,file:///home/hadoop/deltalake/speed_test/raw_d...


In [10]:
plot_bgcolor = "#2c292d"
#paper_bgcolor ="#211f22"
paper_bgcolor ="#1a1d21"
download_color = "#ab9df2"
upload_color = "#78dce8"
default_fontcolor = "white"

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash.dependencies import Input, Output
from dash import dcc, html
import dash_bootstrap_components as dbc
import dash


def line_chart_download_vs_upload(fig,df):

    x = df.log_timestamp


    
    fig.add_trace(
        go.Scatter(
            x=x, y=df.upload_Mbytes, name="Upload (Mbps)", line=dict(color=upload_color)
        ),row =1, col=2,
    )
    fig.update_yaxes(
    title_text="<b>Upload (Mbps)</b>",
    color=upload_color,
    rangemode = 'tozero',
     showgrid=False,
    row =1, col=2,
    )



    fig.add_trace(
        go.Scatter(
            x=x,
            y=df.download_Mbytes,
            name="Download (Mbps)",
            line=dict(color=download_color),

        ),row =2, col=2,
    )

    fig.update_yaxes(
        title_text="<b>Download (Mbps)</b>",
        color=download_color,
        rangemode = 'tozero',
         showgrid=False,
        row =2, col=2,
    )

    fig.update_xaxes(
        showgrid=False,
    )

    fig.update_layout(
        plot_bgcolor=plot_bgcolor,
        paper_bgcolor=paper_bgcolor,
        font=dict(color="white"),
        legend=dict(orientation="h", yanchor="bottom", y=-0.15, xanchor="right", x=1),
    )
   
   


def gauges_indicators(fig,value):

    def gauge_chart(value, steps, title, color):
        max_step = steps[-1][-1]
        title = f"{title} <span style='font-size:0.8em;color:gray'>MBps</span><br><span style='font-size:0.5em;color:gray'>Average</span>"
        gauge = go.Indicator(
                mode="gauge+number+delta",
                value=value,
                domain={"x": [0.25,0.55], "y":[0.25,0.55]},
                title={"text": title, "font": {"size": 25},'align': 'center' },
                delta={"reference": steps[-1][0],  "font": {"size": 13},"increasing": {"color": color}},
                number={"font": {"size": 25}},
                gauge={
                "axis": {
                    "range": [None, max_step],
                    "tickwidth": 2,
                    "tickcolor": plot_bgcolor,
                },
                "bar": {"color": color},
                "bgcolor": "white",
                "borderwidth": 2,
                "bordercolor": "gray",
                "steps": [
                    {"range": steps[0], "color": "#ff6188"},
                    {"range": steps[1], "color": "#fc9867"},
                    {"range": steps[2], "color": "#a9dc76"},
                ],
            },
            )
        
       
        return gauge

    
    fig.add_trace( gauge_chart(
        value["upload_Mbytes"],
        steps=[[0, 10], [10, 15], [15, 20]],
        title=f"<span style='font-size:0.8em;color:{upload_color}'>Upload</span>",
        color=upload_color,
    ),row =1, col=1)

    fig.add_trace(gauge_chart(
        value["download_Mbytes"],
        steps=[[0, 450], [450, 600], [600, 700]],
        title=f"<span style='font-size:0.8em;color:{download_color}'>Download</span>",
        color=download_color,
    )  ,row =2, col=1 )

    fig.update_layout(
            paper_bgcolor=paper_bgcolor,
            font={"color": "white", "family": "Arial"},
            showlegend=False
        )
    fig.update_traces(
        number=dict(font=dict(size=28)),
        delta=dict(font=dict(size=25))
        )

def multiplot_speedtest(df):

    fig = make_subplots(
    rows=2, cols=2,
    specs=[
            [{"type": "domain"}, {} ],
            [ {"type": "domain"}, {}]
    ],
    column_widths = [0.30,0.70],
    row_heights = [0.25,0.25],
    horizontal_spacing=0.15,
    vertical_spacing=0.15,
          )
    
    values = df.iloc[-3:].mean()
    gauges_indicators(fig,values)
    line_chart_download_vs_upload(fig,df)
    fig.update_layout(height=550, margin=dict(l=35,r=35,b=30,t=55))
    
    return fig

    

def register_Callback(app):
    @app.callback(
        Output("stream_line_chart", "figure"),
        [Input("interval-component", "n_intervals"),
        ],
    )
    def streamFig(intervals):

        df = spark.read.table(BRONZE_TABLE).orderBy('log_timestamp',ascending=False).limit(10).toPandas().sort_values('log_timestamp',ascending=True)
        return multiplot_speedtest(df)



config = {"displaylogo": False, "scrollZoom": False, "displayModeBar": False}

updates = dcc.Interval(
    id="interval-component", interval=10000, n_intervals=0  # in milliseconds
)


navbar = dbc.Navbar(
    dbc.Container(
        [
            html.A(
                # Use row and col to control vertical alignment of logo / brand
                dbc.Row(
                    [
                        dbc.Col(html.Img(src="https://www.pinclipart.com/picdir/big/491-4917274_panama-flag-png-palestine-flag-vector-clipart.png", height="30px")),
                        dbc.Col(dbc.NavbarBrand("Network Speed Test by Jose Quesada", className="ms-2")),
                    ],
                    align="center",
                    className="g-0",
                ),
                href="https://plotly.com",
                style={"textDecoration": "none"},
            ),
            dbc.NavbarToggler(id="navbar-toggler", n_clicks=0),
            
        ]
    ),
    color=paper_bgcolor,
    dark=True,
)


streaming_col = dbc.Col(dcc.Graph(id="stream_line_chart", config=config))

layout = dbc.Container(
    
    [ navbar,
    dbc.Container([updates,
                    dcc.Store(id='last_32hrs'),
                                dbc.Row(streaming_col), 
    ],
            style={
                "background-color": paper_bgcolor,
                "color": default_fontcolor
            })])

app = dash.Dash(
    external_stylesheets=[
        "https://cdn.jsdelivr.net/npm/bootstrap@4.3.1/dist/css/bootstrap.min.css"
    ],
)
#app.config.suppress_callback_exceptions = True
app.layout = layout
register_Callback(app)
app.run()

In [11]:
print(1)

1
